In [1]:
import numpy as np      
import matplotlib.pyplot as plt 
import scipy.io.wavfile 
import subprocess
import librosa
import librosa.display
import IPython.display as ipd

from pathlib import Path, PurePath   
from tqdm.notebook import tqdm

## Utility functions

In [2]:
def convert_mp3_to_wav(audio:str) -> str:  
    """Convert an input MP3 audio track into a WAV file.

    Args:
        audio (str): An input audio track.

    Returns:
        [str]: WAV filename.
    """
    if audio[-3:] == "mp3":
        wav_audio = audio[:-3] + "wav"
        if not Path(wav_audio).exists():
                subprocess.check_output(f"ffmpeg -i {audio} {wav_audio}", shell=True)
        return wav_audio
    
    return audio

def plot_spectrogram_and_picks(track:np.ndarray, sr:int, peaks:np.ndarray, onset_env:np.ndarray) -> None:
    """[summary]

    Args:
        track (np.ndarray): A track.
        sr (int): Aampling rate.
        peaks (np.ndarray): Indices of peaks in the track.
        onset_env (np.ndarray): Vector containing the onset strength envelope.
    """
    times = librosa.frames_to_time(np.arange(len(onset_env)),
                            sr=sr, hop_length=HOP_SIZE)

    plt.figure()
    ax = plt.subplot(2, 1, 2)
    D = librosa.stft(track)
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(D), ref=np.max),
                            y_axis='log', x_axis='time')
    plt.subplot(2, 1, 1, sharex=ax)
    plt.plot(times, onset_env, alpha=0.8, label='Onset strength')
    plt.vlines(times[peaks], 0,
            onset_env.max(), color='r', alpha=0.8,
            label='Selected peaks')
    plt.legend(frameon=True, framealpha=0.8)
    plt.axis('tight')
    plt.tight_layout()
    plt.show()

def load_audio_picks(audio, duration, hop_size):
    """[summary]

    Args:
        audio (string, int, pathlib.Path or file-like object): [description]
        duration (int): [description]
        hop_size (int): 

    Returns:
        tuple: Returns the audio time series (track) and sampling rate (sr), a vector containing the onset strength envelope
        (onset_env), and the indices of peaks in track (peaks).
    """
    try:
        track, sr = librosa.load(audio, duration=duration)
        onset_env = librosa.onset.onset_strength(track, sr=sr, hop_length=hop_size)
        peaks = librosa.util.peak_pick(onset_env, 10, 10, 10, 10, 0.5, 0.5)
    except Error as e:
        print('An error occurred processing ', str(audio))
        print(e)

    return track, sr, onset_env, peaks
    
    

## Settings

In [3]:
N_TRACKS = 1413
HOP_SIZE = 512
DURATION = 30 # TODO: to be tuned!
THRESHOLD = 5 # TODO: to be tuned!

In [4]:
data_folder = Path("./data/mp3s-32k/")
mp3_tracks = data_folder.glob("*/*/*.mp3")
tracks = data_folder.glob("*/*/*.wav")

In [5]:
songs = list(data_folder.glob("*/*/*.wav"))

## Preprocessing

In [6]:
def preprocessing(mp3_tracks):
    for track in tqdm(mp3_tracks, total=N_TRACKS):
        convert_mp3_to_wav(str(track))

In [7]:
#preprocessing(mp3_tracks)

## Audio signals

In [8]:
"""for idx, audio in enumerate(tracks):
    if idx >= 1:
        break
    track, sr, onset_env, peaks = load_audio_picks(audio, DURATION, HOP_SIZE)
    plot_spectrogram_and_picks(track, sr, peaks, onset_env)
        
        """

'for idx, audio in enumerate(tracks):\n    if idx >= 1:\n        break\n    track, sr, onset_env, peaks = load_audio_picks(audio, DURATION, HOP_SIZE)\n    plot_spectrogram_and_picks(track, sr, peaks, onset_env)\n        \n        '

## Minhash

In [9]:
# TODO

In [10]:
from bitstring import BitArray
import pandas as pd
from collections import *
import pickle
import multiprocessing
from multiprocessing.dummy import Pool
import random

In [11]:
def save_object(obj, filename):
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)

In [12]:
def read_object(filename):
    with open(filename, 'rb') as file:
        data = pickle.load(file)
    return data

In [13]:
def timeOfPeaks(peaks, times):
    timesPeaks = []
    
    for i in peaks:
        timesPeaks.append(times[i])
    
    return timesPeaks

In [14]:
def fibonacci_hash_float(value:float, rand = False, hash_size = 15):

    value = BitArray(float=value, length=64)
    phi = (1 + 5 ** 0.5) / 2
    g = int(2 ** 64 /phi)


    value ^= value >> 61

    if(rand):
        value = int(g * value.float * np.random.random_sample(1))
    else:
        value = int(g * value.float)

    return int(str(value)[0:hash_size])

In [15]:
class HashTable:
    def __init__(self):
        self.hash_table = defaultdict(list)
        
    def generate_hash(self, inp_vector):
        hashVal = "".join(inp_vector)
        return hashVal
            
    def setitem(self, vec, label):
        val = self.generate_hash(vec)
        self.hash_table[val].append(label)
        
    def setTable(self, dic):
        
        for i in dic:
            self.hash_table[self.generate_hash(dic[i])].append(i)
        
    def getitem(self, inp_vec):
        hash_value = self.generate_hash(inp_vec)
        return self.hash_table.get(hash_value, [])
    
    def getTable(self):
        return(self.hash_table)

In [16]:
class LSH:
    def __init__(self, num_tables, threshold, b):
        assert num_tables == threshold//b, "The number of table must be equals to threshold // band"
        self.num_tables = num_tables
        self.band = b
        self.threshold = threshold
        self.hash_tables = list()
        for i in range(self.num_tables):
            self.hash_tables.append(HashTable())
        
            
    def minhash(self, matrix, labels):
        matrix = np.array(matrix)
        out = defaultdict(lambda: defaultdict(list))
        
        for i in tqdm(range(0,self.threshold)):
            
            if(matrix.shape[0] != 1):

                np.random.shuffle(matrix)

                for col, name in zip(matrix.T, labels):

                    out[i//self.band][name].append((str((col!=0).argmax(axis=0))))

            else:
                np.random.shuffle(matrix.T)

                for idx, el in enumerate(matrix.T):
                    if(el != 0):
                        out[i//self.band][labels[0]].append(str(idx))

                        break

        for key, table in zip(out, self.hash_tables):
            table.setTable(out[key])


    def info(self):
        print("Numero di tabelle: " + str(self.num_tables))
        print("Elementi per tabella: " + str(len(self.hash_tables[0].hash_table)))

In [17]:
def minhash(matrix, labels):
    matrix = np.array(matrix)
    out = defaultdict(lambda: defaultdict(list))

    for i in tqdm(range(0,150)):

        if(matrix.shape[0] != 1):

            np.random.shuffle(matrix)

            for col, name in zip(matrix.T, labels):

                out[i//3][name].append((str((col!=0).argmax(axis=0))))

        else:
            np.random.shuffle(matrix.T)

            for idx, el in enumerate(matrix.T):
                if(el != 0):
                    out[i//3][labels[0]].append(str(idx))

                    break

    return out

In [18]:
#minhash([make_signature_matrix('data/mp3s-32k/radiohead/Amnesiac/06-Knives_Out.wav')],["test"])

In [19]:
#"".join(['22', '1204', '307'])

In [45]:
def make_fingerprints(audio, duration, hop = 0):
    track, sr, onset_env, peaks = load_audio_picks(audio, duration, HOP_SIZE)
    times = librosa.frames_to_time(np.arange(len(onset_env)), sr=sr, hop_length=HOP_SIZE)
    timesPeaks = timeOfPeaks(peaks, times)
    freqsP = [onset_env[i] for i in peaks]
    fingerprints = []
    
    if(hop != 0):
        sec = hop
        time=0
        count=0
        while(sec <= duration):

            idx = 0
            hashVal = 0
            count += 1

            while(time <=sec):

                if(timesPeaks[idx] < sec and timesPeaks[idx] > time):
                    hashVal ^= fibonacci_hash_float(freqsP[idx]) ^ hashVal


                time = timesPeaks[idx]

                if(idx+1 < len(freqsP)):
                    idx += 1
                else:
                    break

            fingerprints.append(hashVal)

            sec += hop

        fingerprints = list(filter(lambda a: a != 0, fingerprints))
    
    else:
        for fr, tm in zip(freqsP, timesPeaks):
            fingerprints.append(fr + tm)
   
    return fingerprints

In [21]:
def make_vocabulary_of_fingerprints(songs):
    
    voc = dict()
    tempList = list()
    
    with Pool(multiprocessing.cpu_count()) as pool:
       
         with tqdm(total = 1413) as pbar:
            for i, el in enumerate(pool.imap(lambda song: make_fingerprints(song, 30, 0.5), songs)):
                tempList.extend(el)
                pbar.update()
                
    for fin in tempList:
        voc[fin] = []
        
    save_object(voc, "./data/vocabulary_of_fingerprints.plk")


In [22]:
def make_signature_matrix(songs,  numberOfSongs = 1413):
    
    tempList = []
    matrix_sig = []
    voc = read_object("./data/vocabulary_of_fingerprints.plk")

    
    if(isinstance(songs, str)):
        
        tempList = make_fingerprints(songs, 30,0.5)
        
        for key in voc:
            voc[key] = list(np.zeros(1, dtype="int"))
        
        for fig in tempList:
            voc[fig].append(1)
            
            
        for k in voc:
            matrix_sig.extend(voc[k])
        
    else:

        for key in voc:
            voc[key] = list(np.zeros(numberOfSongs, dtype="int"))

        with Pool(multiprocessing.cpu_count()) as pool:

            with tqdm(total = numberOfSongs) as pbar:
                for i, el in enumerate(pool.imap(lambda song: make_fingerprints(song, 30, 0.5), songs)):
                    tempList.append(el)
                    pbar.update()


        for idx, song in enumerate(tempList):
            for fin in song:
                voc[fin][idx] = 1
                
        for k in voc:
            matrix_sig.append(voc[k])

        save_object(matrix_sig, "./data/signature_matrix.plk")
        
    del(voc)
    
    return matrix_sig

In [23]:
#make_signature_matrix(songs);

In [24]:
def populate_LSH(songs):
    
    lsh = LSH(10000, 10000, 1)
    
    if(len(songs) != 1):
   
        matrix_sig = read_object("./data/signature_matrix.plk")
        
        lsh.minhash(matrix_sig, songs)
            
    else:
        vector_sig = make_signature_matrix(songs[0])

        lsh.minhash([vector_sig], songs)
        
    return lsh

In [25]:
def searchForCollision(lsh, lsh_q):
    match = list()
    
    for table, tableq in zip(lsh.hash_tables, lsh_q.hash_tables):
        
        match.extend(table.hash_table.get([*tableq.hash_table.keys()][0], ["NA"]))
        
    match = list(filter(lambda a: a != "NA", match))
    
    try:
        prob = Counter(match).most_common(1)[0][1] / len(match)
    except IndexError:
        print("No match found")
        return None
    
    return [Counter(match).most_common(3), prob]

In [26]:
#lsh = populate_LSH(['data/mp3s-32k/radiohead/Amnesiac/06-Knives_Out.wav'])
#save_object(lsh, "./data/lsh_50_150_3(0.3).pkl")
#lsh = read_object("./data/lsh_15_150_10(0.3).pkl")
#lsh_q = populate_LSH(['data/mp3s-32k/radiohead/Amnesiac/06-Knives_Out.wav'])
#out = searchForCollision(lsh, lsh_q)
#out

In [27]:
#len(v)

In [28]:
"""for i in lsh_q.hash_tables:
    print(i.hash_table)"""

'for i in lsh_q.hash_tables:\n    print(i.hash_table)'

In [29]:
"""for k in range(1,200):
    print(1/(4*k))"""

'for k in range(1,200):\n    print(1/(4*k))'

In [30]:
#lsh_q.hash_tables[2].hash_table

In [31]:
audioq = "./data/queries/track1.wav"
audio = "./data/mp3s-32k/aerosmith/Aerosmith/03-Dream_On.wav"

# Query test

take the fisrt query

In [32]:
"""audio = 'data/queries/track3.wav'"""

"audio = 'data/queries/track3.wav'"

make the hashmin of the song

In [33]:
"""track, sr, onset_env, peaks = load_audio_picks(audio, DURATION, HOP_SIZE)
timess = librosa.frames_to_time(np.arange(len(onset_env)), sr=sr, hop_length=HOP_SIZE)
timesPeaks = timeOfPeaks(peaks, timess)
freqsP = [onset_env[i] for i in peaks]
    
h = minhash(freqsP, timesPeaks, THRESHOLD, DURATION)
h"""

'track, sr, onset_env, peaks = load_audio_picks(audio, DURATION, HOP_SIZE)\ntimess = librosa.frames_to_time(np.arange(len(onset_env)), sr=sr, hop_length=HOP_SIZE)\ntimesPeaks = timeOfPeaks(peaks, timess)\nfreqsP = [onset_env[i] for i in peaks]\n    \nh = minhash(freqsP, timesPeaks, THRESHOLD, DURATION)\nh'

lets see if it match something...

In [34]:
"""guess_song(audio)"""

'guess_song(audio)'

In [35]:
"""data_folder2 = Path("./data/queries/")
query_tracks = data_folder2.glob("./*.wav")
get = 0
miss = 0
for query in query_tracks:
    print("\nCurrent query: " + str(query) + "\n")
    try:
        print(guess_song(query))
        get += 1
    except KeyError:
        print("Not matched!")
        miss += 1
    print("\n===========================================\n")
    
print("Song matched: " + str(get) + "  Song missed: " + str(miss))"""

'data_folder2 = Path("./data/queries/")\nquery_tracks = data_folder2.glob("./*.wav")\nget = 0\nmiss = 0\nfor query in query_tracks:\n    print("\nCurrent query: " + str(query) + "\n")\n    try:\n        print(guess_song(query))\n        get += 1\n    except KeyError:\n        print("Not matched!")\n        miss += 1\n    print("\n===========================================\n")\n    \nprint("Song matched: " + str(get) + "  Song missed: " + str(miss))'

In [36]:
import random, copy, struct
import warnings
import numpy as np


class MinHash(object):

    _mersenne_prime = np.uint64((1 << 61) - 1)
    _max_hash = np.uint64((1 << 32) - 1)
    _hash_range = (1 << 32)

    def __init__(self, num_perm=128, seed=1, hashfunc=fibonacci_hash_float):

        self.seed = seed
        self.num_perm = num_perm
        self.hashfunc = hashfunc
        self.hashvalues = self._init_hashvalues(num_perm)
        self.permutations = self._init_permutations(num_perm)


    def _init_hashvalues(self, num_perm):
        return np.ones(num_perm, dtype=np.uint64)*_max_hash

    def _init_permutations(self, num_perm):
        # Create parameters for a random bijective permutation function
        # that maps a 32-bit hash value to another 32-bit hash value.
        # http://en.wikipedia.org/wiki/Universal_hashing
        gen = np.random.RandomState(self.seed)
        return np.array([
            (gen.randint(1, _mersenne_prime, dtype=np.uint64), gen.randint(0, _mersenne_prime, dtype=np.uint64)) for _ in range(num_perm)
        ], dtype=np.uint64).T



    def update_batch(self, hv):
        #hv = np.array([self.hashfunc(_b) for _b in b], dtype=np.uint64)
        hv = np.array(hv, dtype=np.uint64)
        a, b = self.permutations
        phv = np.bitwise_and(((hv * np.tile(a, (len(hv), 1)).T).T + b) % _mersenne_prime, _max_hash)
        self.hashvalues = np.vstack([phv, self.hashvalues]).min(axis=0)

    def jaccard(self, other):
        return float(np.count_nonzero(self.hashvalues==other.hashvalues)) / float(len(self.hashvalues))

In [37]:
def make_all_fingerprints(): 
    tempList = list()
    with Pool(multiprocessing.cpu_count()) as pool:
       
         with tqdm(total = 1413) as pbar:
            for i, el in enumerate(pool.imap(lambda song: make_fingerprints(song, 120), songs)):
                tempList.append(el)
                pbar.update()
                
    return tempList

In [38]:
from datasketch.hashfunc import sha1_hash32

In [39]:
import datasketch

In [52]:
#fings = make_all_fingerprints()
fings = read_object("./data/figs.plk")
minhashes = []

for fing, song in zip(fings,songs):
    m = datasketch.MinHash()
    m.update_batch(fing)
    minhashes.append((m,song))
    


In [53]:
mq = datasketch.MinHash()

mq.update_batch(make_fingerprints("./data/queries/track1.wav", duration=100))

In [54]:
out = []
for h in minhashes:
    out.append((h[0].jaccard(mq), h[1]))

sorted(out, reverse=True)

[(0.84375, PosixPath('data/mp3s-32k/aerosmith/Aerosmith/03-Dream_On.wav')),
 (0.0, PosixPath('data/mp3s-32k/u2/Zooropa/10-The_Wanderer.wav')),
 (0.0, PosixPath('data/mp3s-32k/u2/Zooropa/09-Dirty_Day.wav')),
 (0.0, PosixPath('data/mp3s-32k/u2/Zooropa/08-The_First_Time.wav')),
 (0.0,
  PosixPath('data/mp3s-32k/u2/Zooropa/07-Some_Days_Are_Better_Than_Others.wav')),
 (0.0,
  PosixPath('data/mp3s-32k/u2/Zooropa/06-Daddy_s_Gonna_Pay_For_Your_Crashed_Car.wav')),
 (0.0, PosixPath('data/mp3s-32k/u2/Zooropa/05-Stay_Faraway_So_Close_.wav')),
 (0.0, PosixPath('data/mp3s-32k/u2/Zooropa/04-Lemon.wav')),
 (0.0, PosixPath('data/mp3s-32k/u2/Zooropa/03-Numb.wav')),
 (0.0, PosixPath('data/mp3s-32k/u2/Zooropa/02-Babyface.wav')),
 (0.0, PosixPath('data/mp3s-32k/u2/Zooropa/01-Zooropa.wav')),
 (0.0, PosixPath('data/mp3s-32k/u2/War/10-_40_.wav')),
 (0.0, PosixPath('data/mp3s-32k/u2/War/09-Surrender.wav')),
 (0.0, PosixPath('data/mp3s-32k/u2/War/08-Red_Light.wav')),
 (0.0, PosixPath('data/mp3s-32k/u2/War/07-Tw